In [2]:
import logging
from pprint import pprint
from sys import stdout as STDOUT


# Example 1
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    def get_private_field(self):
        return self.__private_field


# Example 2
foo = MyObject()
assert foo.public_field == 5


# Example 3
assert foo.get_private_field() == 10



In [3]:
# Example 4
try:
    foo.__private_field
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-3-0937af290866>", line 3, in <module>
    foo.__private_field
AttributeError: 'MyObject' object has no attribute '__private_field'


In [4]:
# Example 5
class MyOtherObject(object):
    def __init__(self):
        self.__private_field = 71

    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

In [5]:
# Example 6
try:
    class MyParentObject(object):
        def __init__(self):
            self.__private_field = 71
    
    class MyChildObject(MyParentObject):
        def get_private_field(self):
            return self.__private_field
    
    baz = MyChildObject()
    baz.get_private_field()
except:
    logging.exception('Expected')
else:
    assert False



ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-5-5b70d19a27b0>", line 12, in <module>
    baz.get_private_field()
  File "<ipython-input-5-5b70d19a27b0>", line 9, in get_private_field
    return self.__private_field
AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'


{'_MyParentObject__private_field': 71}


In [6]:

# Example 7
assert baz._MyParentObject__private_field == 71


# Example 8
print(baz.__dict__)


{'_MyParentObject__private_field': 71}


In [7]:
# Example 9
class MyClass(object):
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return str(self.__value)

foo = MyClass(5)
assert foo.get_value() == '5'



The __value attribute is now assigned in the MyBaseClass parent class, not the
MyClass parent. That causes the private variable reference self._MyClass__value
to break in MyIntegerSubclass.

In [8]:
# Example 10
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

foo = MyIntegerSubclass(5)
assert foo.get_value() == 5



In general, it’s better to err on the side of allowing subclasses to do more by using
protected attributes. Document each protected field and explain which are internal APIs
available to subclasses and which should be left alone entirely. This is as much advice to
other programmers as it is guidance for your future self on how to extend your own code
safely.

The only time to seriously consider using private attributes is when you’re worried about
naming conflicts with subclasses. This problem occurs when a child class unwittingly
defines an attribute that was already defined by its parent class.


In [9]:

# Example 11
class MyBaseClass(object):
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return self.__value

class MyClass(MyBaseClass):
    def get_value(self):
        return str(super().get_value())

class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

In [10]:
# Example 12
try:
    foo = MyIntegerSubclass(5)
    foo.get_value()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-10-069ebd71a80b>", line 4, in <module>
    foo.get_value()
  File "<ipython-input-9-5cd9ff56c9f1>", line 16, in get_value
    return int(self._MyClass__value)
AttributeError: 'MyIntegerSubclass' object has no attribute '_MyClass__value'


In [12]:
# Example 13
class MyClass(object):
    def __init__(self, value):
        # This stores the user-supplied value for the object.
        # It should be coercible to a string. Once assigned for
        # the object it should be treated as immutable.
        self._value = value

    def get_value(self):
        return str(self._value)
    
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return self._value

foo = MyIntegerSubclass(5)
assert foo.get_value() == 5


This is primarily a concern with classes that are part of a public API; the subclasses are
out of your control, so you can’t refactor to fix the problem. Such a conflict is especially
possible with attribute names that are very common (like value). To reduce the risk of
this happening, you can use a private attribute in the parent class to ensure that there are
no attribute names that overlap with child classes.

In [13]:
# Example 14
class ApiClass(object):
    def __init__(self):
        self._value = 5

    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # Conflicts

a = Child()
print(a.get(), 'and', a._value, 'should be different')

hello and hello should be different


In [14]:
# Example 15
class ApiClass(object):
    def __init__(self):
        self.__value = 5

    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # OK!

a = Child()
print(a.get(), 'and', a._value, 'are different')

5 and hello are different


* Private attributes aren’t rigorously enforced by the Python compiler.
* Plan from the beginning to allow subclasses to do more with your internal APIs and attributes instead of locking them out by default.
* Use documentation of protected fields to guide subclasses instead of trying to force access control with private attributes.
* Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control.